Jupyter is now (2016-06-20) supported by Google Datalabs
# Hands on example of making predictive models

# Customer Churn

"Churn Rate" - measure of the number of individuals or items moing out of a collective group over a specific period of time. - Wikipedia

<img src="https://www.statuscake.com/wp-content/uploads/2016/02/Customer-Churn.jpg" style="width:70%" />

Churn (loss of customers to competition) is a problem for any company because it is more expensive to acquire a new customer than to keep your existing one from leaving. This example we will examine the churn for telecom companies.

## Binary Classification

More on [classification](data-mining-intro-ML-may-2016.ipynb) in the machine learning intro notebook.


This labeling schema is arguably the most common classification schema which is called binary classification as I have only two classes to predict(churn or not). Even some of the multi-class classification problems could be expressed as a binary classification schema(one vs other classes). 

<img src="http://www.holehouse.org/mlclass/06_Logistic_Regression_files/Image%20[23].png" style="width:70%"/>

Therefore, it is a powerful/basic classification schema that you would use approximately 83%(put some random number > 50) of your machine learning problems.

## Example of real application

Most telecom companies suffer from voluntary churn. Churn rate has strong impact on the life time value of the customer because it affects the length of service and the future revenue of the company. 

* For example if a company has **25% churn rate** then the average customer lifetime is **4 years**; 
* similarly a company with a **churn rate of 50%**, has an average customer lifetime of **2 years**. 

It is estimated that 75 percent of the 17 to 20 million subscribers signing up with a new wireless carrier every year are coming from another wireless provider, which means they are churners. Telecom companies spend hundreds of dollars to acquire a new customer and when that customer leaves, the company not only loses the future revenue from that customer but also the resources spend to acquire that customer. Churn erodes profitability.


Steps that have been adopted by telecom companies so far:

Telecom companies have used two approaches to address churn - 
* **Untargeted approach.** The untargeted approach relies on superior product and mass advertising to increase brand loyalty and thus retain customers. 
* **Targeted approach.** The targeted approach relies on identifying customers who are likely to churn, and provide intervention to encourage them.

---

So what are we going to predict? We ask the business about the problem and what question there is to answer.

<img src="../res/fig/questions.png" style="width:50%">

## Imports

In [ ]:
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

from sklearn.cross_validation import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RF
%matplotlib inline

## Preparing data / Data Mining / Data crunching / Cleaning data

<img src="https://icrunchdatanews.com/wp-content/uploads/2015/08/Process-getting-and-cleaning-data-v3.jpg" >

To clean data takes time. 80 % of the time is spent on cleaning the data.

### Two rules of data cleaning

1. Data never comes in the format you want
2. Assuming that at least 5 % of the data is dirty

---

### Loading data from the CSV

The data is straightforward. Each row represents a subscribing telephone customer. Each column contains customer attributes such as phone number, call minutes used during different times of day, charges incurred for services, lifetime account duration, and whether or not the customer is still a customer.

In [ ]:
churn_df = pd.read_csv('../data/raw/churn.csv')
col_names = churn_df.columns.tolist()

print("Columns names:",col_names)

print("\n Sample data:")
to_show = col_names[:6] + col_names[-6:]
churn_df[to_show].head(6)

In [ ]:
# Isolate target data
churn_result = churn_df['Churn?']
y = np.where(churn_result == 'True.',1,0)

# remove some columns, we might need them 
# but this is just for example purpose
to_drop = ['State', 'Area Code', 'Phone', 'Churn?']
churn_feat_space = churn_df.drop(to_drop, axis=1)

# 'yes'/'no' has to be converted to boolean values
# NumPy converts these from boolean to 1. and 0. later
yes_no_cols = ["Int'l Plan","VMail Plan"]
churn_feat_space[yes_no_cols] = churn_feat_space[yes_no_cols] == 'yes'

In [ ]:
# remove the test person from the training set
churn_feat_space.drop([-2:-1])

test_person = churn_feat_space[-2:-1]
test_person

In [ ]:
expected_test_person_churn = y[-1]
print(expected_test_person_churn)

#### Preparing the training data for the models

Many predictors care about the relative size of different features even though those scales might be arbitrary. For instance: the number of points a basketball team scores per game will naturally be a couple orders of magnitude larger than their win percentage. But this doesn't mean that the latter is 100 times less signifigant. 


StandardScaler fixes this by normalizing each feature to a range of around 1.0 to -1.0 thereby preventing models from misbehaving. Well, at least for that reason.

Great, I now have a feature space X and a set of target values y. On to the predictions!

In [ ]:
churn_feat_space = churn_feat_space.fillna(0)
X = churn_feat_space.as_matrix().astype(np.float)

scaler = StandardScaler()
X = scaler.fit_transform(X)
print("Feature space holds %d observations and %d features" % X.shape)
print("Unique target labels:", np.unique(y))

### Simple classifier
**ALWAYS** start **SIMPLE.**

Here we make support vector machine classifer to begin with.

In [ ]:
# clf stands for classifier and SVC stands for support vector machine
clf = SVC(kernel='rbf')
clf = clf.fit(X,y)
print("Score",clf.score(X,y))

Predicting the outcome of Ms. Test Person

In [ ]:
print(clf.predict(test_person))

<img src="http://i.imgur.com/lmmBt.gif" >

## Model the domain, but how good is your model?
Express, test, cycle. A machine learning pipeline should be anything but static. There are always new features to design, new data to use, new classifiers to consider each with unique parameters to tune. And for every change it's critical to be able to ask, "Is the new version better than the last?" So how do I do that?

As a good start, cross validation will be used throught this example. Cross validation attempts to avoid overfitting (training on and predicting the same datapoint) while still producing a prediction for each observation dataset. This is accomplished by systematically hiding different subsets of the data while training a set of models. After training, each model predicts on the subset that had been hidden to it, emulating multiple train-test splits. When done correctly, every observation will have a 'fair' corresponding prediction.

Here's what that looks like using scikit-learn libraries.


## Cross Validation
Pretty much tries to fit model parameters for the data at hand

In [ ]:
from sklearn.cross_validation import KFold

def run_cv(X,y,clf_class,**kwargs):
    # Construct a kfolds object
    kf = KFold(len(y),n_folds=3,shuffle=True)
    y_pred = y.copy()
    
    # Iterate through folds
    for train_index, test_index in kf:
        X_train, X_test = X[train_index], X[test_index]
        y_train = y[train_index]
        # Initialize a classifier with key word arguments
        clf = clf_class(**kwargs)
        clf.fit(X_train,y_train)
        y_pred[test_index] = clf.predict(X_test)
    return y_pred

Let's compare three fairly unique algorithms support vector machines, random forest, and k-nearest-neighbors. Nothing fancy here, just passing each to cross validation and determining how often the classifier predicted the correct class.

* **Support Vector Machines**
* **Random Forest**
* **K-nearest-neighbors**

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.metrics import average_precision_score

def accuracy(y_true,y_pred):
    # NumPy interpretes True and False as 1. and 0.
    return np.mean(y_true == y_pred)

print("Accuracy",)
print("Logistic Regression:")
print("%.3f" % accuracy(y, run_cv(X,y,LR)))
print("Gradient Boosting Classifier")
print("%.3f" % accuracy(y, run_cv(X,y,GBC)))
print("Support vector machines:")
print("%.3f" % accuracy(y, run_cv(X,y,SVC)))
print("Random forest:")
print("%.3f" % accuracy(y, run_cv(X,y,RF)))
print("K-nearest-neighbors:")
print("%.3f" % accuracy(y, run_cv(X,y,KNN)))


**Random forest** won, right?

## Precision and recall 

Measurements aren't golden formulas which always spit out high numbers for good models and low numbers for bad ones. Inherently they convey something sentiment about a model's performance, and it's the job of the human designer to determine each number's validity. The problem with accuracy is that outcomes aren't necessarily equal. If my classifier predicted a customer would churn and they didn't, that's not the best but it's forgivable. However, if my classifier predicted a customer would return, I didn't act, and then they churned... that's really bad.

We'll be using another built in `scikit-learn` function to construction a confusion matrix. A confusion matrix is a way of visualizing predictions made by a classifier and is just a table showing the distribution of predictions for a specific class. The x-axis indicates the true class of each observation (if a customer churned or not) while the y-axis corresponds to the class predicted by the model (if my classifier said a customer would churned or not).

## Confusion matrix and confusion tables: 
The columns represent the actual class and the rows represent the predicted class. Lets evaluate performance: 

|      | Condition True | Condition False|
|------|----------------|---------------|
|**Prediction True**|True Positive|False positive|
|**Prediction False**|False Negative|True Negative|

Sensitivity, Recall or True Positive Rate quantify the models ability to predict our positive classes. 

$$Recall = \frac{ TP}{TP + FN}$$ 

Specificity or True Negative Rate quantify the models ability to predict our Negative classes. 

$$Precision = \frac{ TN}{FP + TN}$$ 

### Example:

|      | Spam | Ham|
|------|----------------|---------------|
|prediction Spam|100|50|
|Prediction Ham|75|900|

$$Recall = \frac{100}{100 + 75} = 57.14 \%   Sensitive $$

$$Precision = \frac{ 900}{50 + 900} = 94.73 \% Specific $$

Precision is useful for presenting the user with recommended items. How many selected items are relevant?

Recall, how many relevant items are selected?


In [ ]:
from sklearn import metrics
grad_ens_conf_matrix = metrics.confusion_matrix(y, run_cv(X, y, GBC))
svm_svc_conf_matrix = metrics.confusion_matrix(y, run_cv(X, y, SVC))
random_forest_conf_matrix = metrics.confusion_matrix(y, run_cv(X, y, RF))
k_neighbors_conf_matrix = metrics.confusion_matrix(y, run_cv(X, y, KNN))
logistic_reg_conf_matrix = metrics.confusion_matrix(y, run_cv(X, y, LR))
dumb_conf_matrix = metrics.confusion_matrix(y, [0 for ii in y.tolist()]); # ignore the warning as they are all 0

conf_matrix = {
                1: {
                    'matrix': grad_ens_conf_matrix,
                    'title': 'Gradient Boosting',
                   },
                2: {
                    'matrix': svm_svc_conf_matrix,
                    'title': 'Support Vector Machine',
                   },
                3: {
                    'matrix': random_forest_conf_matrix,
                    'title': 'Random Forest',
                   },
                4: {
                    'matrix': k_neighbors_conf_matrix,
                    'title': 'K Nearest Neighbors',
                   },
                5: {
                    'matrix': logistic_reg_conf_matrix,
                    'title': 'Logistic Regression',
                   },
                6: {
                    'matrix': dumb_conf_matrix,
                    'title': 'Dumb',
                   },
}

In [ ]:
import seaborn as sns
plt.style.use('fivethirtyeight') # Good looking plots
fix, ax = plt.subplots(figsize=(16, 12))
plt.suptitle('Confusion Matrix of Various Classifiers')
for ii, values in conf_matrix.items():
    matrix = values['matrix']
    title = values['title']
    plt.subplot(3, 2, ii) # starts from 1
    plt.title(title);
    sns.heatmap(matrix, annot=True,  fmt='');

An important question to ask might be, When an individual churns, how often does my classifier predict that correctly? This measurement is called "recall" and a quick look at these diagrams can demonstrate that random forest is clearly best for this criteria. Out of all the churn cases (outcome "1") random forest correctly retrieved 330 out of 482. This translates to a churn "recall" of about 68% (330/482≈2/3), far better than support vector machines (≈50%) or k-nearest-neighbors (≈35%).

Another question of importance is "precision" or, When a classifier predicts an individual will churn, how often does that individual actually churn? The differences in sematic are small from the previous question, but it makes quite a different. Random forest again out preforms the other two at about 93% precision (330 out of 356) with support vector machines a little behind at about 87% (235 out of 269). K-nearest-neighbors lags at about 80%.

While, just like accuracy, precision and recall still rank random forest above SVC and KNN, this won't always be true. When different measurements do return a different pecking order, understanding the values and tradeoffs of each rating should effect how you proceed.



# Visualizing via a Decision Tree to explore the features!!

In [ ]:
from __future__ import print_function

import os
import subprocess

import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier, export_graphviz

In [ ]:
def visualize_tree(tree, feature_names):
    """Create tree png using graphviz.

    Args
    ----
    tree -- scikit-learn DecsisionTree.
    feature_names -- list of feature names.
    """
    with open("dt.dot", 'w') as f:
        export_graphviz(tree, out_file=f,
                        feature_names=feature_names)

    command = ["dot", "-Tpng", "dt.dot", "-o", "dt.png"]
    try:
        subprocess.check_call(command)
    except:
        exit("Could not run dot, ie graphviz, to "
             "produce visualization")

In [ ]:
visualize_tree(clf,churn_feat_space.columns)

## Deployment

Now that we have a model, we deploy it. Then we need to make a metric of how good it performs to helps us feedback to make it even better. Monitor and creating a feedback loop takes time and effort.

<img src="http://image.slidesharecdn.com/wikujib2qoecc5r7vb7f-signature-87404360c94bb2c35661cdc792a68a5997906369b9fee3d51c5556b0e68e8418-poli-150724043821-lva1-app6891/95/production-and-beyond-deploying-and-managing-machine-learning-models-6-638.jpg?cb=1442685173" >
